# Portfolio Analyze Project

In [1]:
!pip install alpaca-trade-api

In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools1 import MCSimulation

%matplotlib inline

In [3]:
# Load .env enviroment variables
load_dotenv()

True

## Part 1 - Get API data

### Collect Crypto Prices Using the `requests` Library

In [5]:
# Set current amount of crypto assets
nomics_api = os.getenv('NOMICS')
crypto = requests.get('https://api.nomics.com/v1/currencies/ticker?key='+ nomics_api +'&ids=BTC,ETH,XRP&interval=1d,30d&convert=EUR&per-page=100&page=1').json()
crypto

[{'id': 'BTC',
  'currency': 'BTC',
  'symbol': 'BTC',
  'name': 'Bitcoin',
  'logo_url': 'https://s3.us-east-2.amazonaws.com/nomics-api/static/images/currencies/btc.svg',
  'status': 'active',
  'price': '15094.21439174',
  'price_date': '2020-12-09T00:00:00Z',
  'price_timestamp': '2020-12-09T02:27:00Z',
  'circulating_supply': '18566025',
  'max_supply': '21000000',
  'market_cap': '280239561752',
  'num_exchanges': '371',
  'num_pairs': '46293',
  'num_pairs_unmapped': '4696',
  'first_candle': '2011-08-18T00:00:00Z',
  'first_trade': '2011-08-18T00:00:00Z',
  'first_order_book': '2017-01-06T00:00:00Z',
  'rank': '1',
  'rank_delta': '0',
  'high': '16451.53841291',
  'high_timestamp': '2020-11-30T00:00:00Z',
  '1d': {'volume': '24497675120.68',
   'price_change': '-751.61349402',
   'price_change_pct': '-0.0474',
   'volume_change': '5550622970.98',
   'volume_change_pct': '0.2930',
   'market_cap_change': '-13940498900.17',
   'market_cap_change_pct': '-0.0474'},
  '30d': {'volum

In [6]:
btc_price = crypto[0]['price']
eth_price = crypto[1]['price']
xrp_price = crypto[2]['price']
print(btc_price, eth_price, xrp_price)
d = {'BTC': [float(btc_price)], 'ETH': [float(eth_price)], 'XRP': [float(xrp_price)]}
crypto_data = pd.DataFrame(data=d)
crypto_data

15094.21439174 453.39988754 0.45719268


### Collect Investments Data Using Alpaca: stocks, bonds and S&P

In [8]:
# Set Alpaca API key and secret
alpaca_api = os.getenv('ALPACA_API_KEY')
alpaca_secret = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
api = tradeapi.REST(alpaca_api, alpaca_secret, api_version='v2')

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\murip\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [10]:
# Format current date as ISO format
current_date = pd.Timestamp('2020-12-10', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["KO", "PFE", 'O', 'GILD', 'D', 'MAIN', 'UBER', 
           'PINS', 'WKHS', 'CTSH']

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for stocks
ticker_data = api.get_barset(tickers, timeframe, start=current_date).df

# Preview DataFrame
ticker_data.head()

D                                GILD         \
                            open  high    low  close   volume   open   high   
2020-12-08 00:00:00-05:00  75.62  75.8  74.59  74.74  7660059  60.56  61.25   

                                                       KO                 \
                               low  close   volume   open   high     low   
2020-12-08 00:00:00-05:00  60.5294  61.17  4932735  52.94  53.48  52.785   

                                            MAIN                         \
                           close   volume   open     high    low  close   
2020-12-08 00:00:00-05:00  53.19  8369211  31.65  31.9199  31.54  31.67   

                                       O                                PFE  \
                           volume   open   high    low close   volume  open   
2020-12-08 00:00:00-05:00  305520  60.67  61.12  60.39  60.4  1706593  41.4   

                                                           PINS               \
                            high    low  close    volume   open  high    low   
2020-12-08 00:00:00-05:00  42.77  41.35  42.55  80661298  70.25  70.8  68.42   

                                           UBER                                \
                           close   volume  open   high    low close    volume   
2020-12-08 00:00:00-05:00  69.93  5780959  52.4  54.22  52.35  53    23044693   

                            WKHS                                     
                            open     high    low    close    volume  
2020-12-08 00:00:00-05:00  21.06  21.4599  20.17  21.3775  11073677

In [14]:
# Pick close prices
new_data = pd.concat([ticker_data['KO']['close'], ticker_data['PFE']['close'], 
                      ticker_data['O']['close'], ticker_data['GILD']['close'], 
                      ticker_data['D']['close'], ticker_data['MAIN']['close'],
                      ticker_data['UBER']['close'], ticker_data['PINS']['close'],
                      ticker_data['WKHS']['close'], ticker_data['CTSH']['close']],
                     axis=1)
new_data

In [94]:
new_data.columns = ['KO', 'PFE', 'O', 'GILD', 'D', 'MAIN', 'UBER', 
                    'PINS', 'WKHS', 'CTSH']
new_data

,close
2020-12-08 00:00:00-05:00,53.19
2020-12-08 00:00:00-05:00,42.55
2020-12-08 00:00:00-05:00,60.40
2020-12-08 00:00:00-05:00,61.17
2020-12-08 00:00:00-05:00,74.74
2020-12-08 00:00:00-05:00,31.67


In [ ]:
# Setting bonds
bond_tickers = ['TLT', 'BLV', 'IGLB']

# Get current closing prices for stocks
ticker_bond_data = api.get_barset(bond_tickers, timeframe, start=current_date).df

# Preview DataFrame
ticker_bond_data.head()

In [ ]:
# Selecting closing price for bonds
close_bonds = pd.concat([ticker_bond_data['BLV']['close'], ticker_bond_data['IGLB']['close'], 
                         ticker_bond_data['TLT']['close']], axis=1)
close_bonds.columns = ['BLV', 'IGLB', 'TLT']
close_bonds

In [ ]:
# Get current closing prices for SP
sp500_data = pd.read_csv('sp500_history.csv', index_col='Date', parse_dates=True, 
                         infer_datetime_format=True)

# Preview DataFrame
sp500_data.head()

In [ ]:
# Compute the current value of shares
bonds_investment = 10000
crypto_investment = 20000
stocks_investment = 20000

In [ ]:
#We'll be using weights in our project, with the initial investment 
investment_per_stock = round(stocks_investment / len(new_data.columns),2)
investment_per_stock

investment_per_crypto = round(crypto_investment / len(crypto_data.columns),2)
investment_per_crypto

investment_per_bond = round(bonds_investment / len(close_bonds.columns),2)
investment_per_bond

print(investment_per_stock, investment_per_crypto, investment_per_bond)

In [ ]:
# Get stock value, equal investments
stock_value = new_data * investment_per_stock
stock_value

In [ ]:
# Get crypto value, equal investment
crypto_value = crypto_data * crypto_data
crypto_value

## Sharpe Ratios

In [ ]:
# Historical crypto data
current_date = pd.Timestamp('2015-12-10', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-12-10', tz='America/New_York').isoformat()

btc_data = pd.read_csv('BTC.csv', parse_dates=True, infer_datetime_format=True)
eth_data = pd.read_csv('ETH.csv', parse_dates=True, infer_datetime_format=True)
xrp_data = pd.read_csv('XRP.csv', parse_dates=True, infer_datetime_format=True)

btc_data = btc_data.set_index('Date')
btc_data.head()

In [ ]:
eth_data = eth_data.set_index('Date')
eth_data.head()

In [ ]:
xrp_data = xrp_data.set_index('Date')
xrp_data.head()

In [ ]:
#get pct change only
all_crypto = pd.concat([btc_data['Adj Close'], eth_data['Adj Close'], xrp_data['Adj Close']],
                      axis=1, join='inner')
all_crypto.columns = ['BTC Adj Close', 'ETH Adj Close', 'XRP Adj Close']
all_crypto.head()

In [ ]:
##get pct change only
crypto_pct_change = all_crypto.pct_change().dropna()
crypto_pct_change.head()

In [ ]:
# Historical bonds data
# Set the tickers
tickers_bonds = ['BLV', 'IGLB', 'TLT']

# Set timeframe to '1D' for Alpaca API

bonds_historical = api.get_barset(tickers_bonds, timeframe, start=current_date, end=end_date,).df
bonds_historical.head()

In [ ]:
# Select only close prices for bonds
bonds_close_only = pd.concat([bonds_historical['BLV']['close'], 
                              bonds_historical['IGLB']['close'], 
                              bonds_historical['TLT']['close']], axis=1)
bonds_close_only.columns = ['BLV', 'IGLB', 'TLT']
bonds_close_only.head()

In [ ]:
bonds_returns = bonds_close_only.pct_change().dropna()
bonds_returns.head()

In [ ]:
# Removing time from DateTime
bonds_returns.index = pd.to_datetime(bonds_returns.index).date
bonds_returns.index = 'Date'
bonds_returns.head()

In [ ]:
# Historical stock data
# Set the tickers
tickers = ["KO", "PFE", 'O', 'GILD', 'D', 'MAIN', 'UBER', 
           'PINS', 'WKHS', 'CTSH']

# Set timeframe to '1D' for Alpaca API

stock_historical = api.get_barset(tickers, timeframe, start=current_date, end=end_date,).df

stock_historical.head()

In [ ]:
# Select only close prices
stocks_close_only = pd.concat([stock_historical['KO']['close'], stock_historical['PFE']['close'], stock_historical['O']['close'], stock_historical['GILD']['close'], stock_historical['D']['close'], stock_historical['MAIN']['close'],
                     stock_historical['UBER']['close'], stock_historical['PINS']['close'], stock_historical['WKHS']['close'], stock_historical['CTSH']['close']], 
                    axis=1)
stocks_close_only.columns = ['KO', 'PFE', 'O', 'GILD', 'D', 'MAIN', 'UBER', 'PINS', 'WKHS', 'CTSH']
stocks_close_only.head()

In [ ]:
stock_returns = stocks_close_only.pct_change().dropna()
stock_returns.head()

In [ ]:
stock_returns.index = pd.to_datetime(stock_returns.index).date
stock_returns.index = 'Date'
stock_returns.head()

In [ ]:
sharpe_stocks = (stock_returns.mean()*252)/(stock_returns.std() * np.sqrt(252))
sharpe_stocks.sort_values(inplace=True)

In [ ]:
sharpe_stocks.plot(kind='bar', title='Sharpe Ratios for Stocks')

In [ ]:
# Sharpes for bonds
sharpe_bonds = (bonds_returns.mean()*252)/(bonds_returns.std()*np.sqrt(252))
sharpe_bonds.sort_values(inplace=True)

In [ ]:
sharpe_bonds.plot(kind='bar', title='Sharpe Ratios for Bonds')

In [ ]:
# Sharpe for crypto
sharpe_crypto = (crypto_pct_change.mean()*365)/(crypto_pct_change.std()*np.sqrt(365))
sharpe_crypto.sort_values(inplace=True)

In [ ]:
sharpe_crypto.plot(kind='bar', title='Sharpe Ratios for Crypto')

## Part 2 - Monte Carlo Simulation

In [ ]:
#Exclude weekends for crypto data

In [ ]:
#put all data in a single df
pct_change_all = pd.concat([stock_returns, bonds_returns, crypto_returns],
                           join='inner', axis=1)
fund_data

In [1]:
#the sum of the weights must equal 1
weight_crypto = 0.3
weight_stocks = 0.4
weight_bonds = 0.2
weight_options =0.1
fund_weights = [weight_options, weight_bonds, 
                weight_stocks, weight_crypto]

weight_total = sum(fund_weights)

if weight_total != 1:
    print('The sum of the weights must equal 1')
else: 
    print('The sum of the weights equals 1 great job!')

The sum of the weights equals 1 great job!


In [14]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2015-08-07', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-08-07', tz='America/New_York').isoformat()

In [15]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
MC_15_returns = MCSimulation(
    fund_data = fund_data,
    weights = fund_weights,
    num_simulation = 1000,
    num_trading_days = 252*15)

# Print the simulation input data
MC_15_returns.fund_data.head()

AGG                                        \
                             open      high      low   close     volume   
2015-08-07 00:00:00-04:00  109.14  109.2750  109.035  109.21  2041167.0   
2015-08-10 00:00:00-04:00  109.15  109.1700  108.920  109.06  1149778.0   
2015-08-11 00:00:00-04:00  109.42  109.5765  109.284  109.42  1420907.0   
2015-08-12 00:00:00-04:00  109.55  109.7100  109.350  109.36  1468979.0   
2015-08-13 00:00:00-04:00  109.36  109.3651  109.110  109.15  1465173.0   

                              SPY                                     
                             open    high     low   close     volume  
2015-08-07 00:00:00-04:00  208.16  208.34  206.87  207.93   87669782  
2015-08-10 00:00:00-04:00  209.28  210.67  209.28  210.58   66755890  
2015-08-11 00:00:00-04:00  208.98  209.47  207.76  208.63   88424557  
2015-08-12 00:00:00-04:00  207.11  209.14  205.36  208.89  136171450  
2015-08-13 00:00:00-04:00  208.73  209.55  208.01  208.63   77197796

In [16]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
MC_15_returns.calc_cumulative_return()

In [17]:
# Printing the simulation input data
fund_plot = MC_15_returns.plot_simulation()

AGG                                        \
                             open      high      low   close     volume   
2015-08-07 00:00:00-04:00  109.14  109.2750  109.035  109.21  2041167.0   
2015-08-10 00:00:00-04:00  109.15  109.1700  108.920  109.06  1149778.0   
2015-08-11 00:00:00-04:00  109.42  109.5765  109.284  109.42  1420907.0   
2015-08-12 00:00:00-04:00  109.55  109.7100  109.350  109.36  1468979.0   
2015-08-13 00:00:00-04:00  109.36  109.3651  109.110  109.15  1465173.0   

                                           SPY                          \
                          daily_return    open    high     low   close   
2015-08-07 00:00:00-04:00          NaN  208.16  208.34  206.87  207.93   
2015-08-10 00:00:00-04:00    -0.001374  209.28  210.67  209.28  210.58   
2015-08-11 00:00:00-04:00     0.003301  208.98  209.47  207.76  208.63   
2015-08-12 00:00:00-04:00    -0.000548  207.11  209.14  205.36  208.89   
2015-08-13 00:00:00-04:00    -0.001920  208.73  209.55  208.01  208.63   

                                                   
                              volume daily_return  
2015-08-07 00:00:00-04:00   87669782          NaN  
2015-08-10 00:00:00-04:00   66755890     0.012745  
2015-08-11 00:00:00-04:00   88424557    -0.009260  
2015-08-12 00:00:00-04:00  136171450     0.001246  
2015-08-13 00:00:00-04:00   77197796    -0.001245

In [18]:
# Running a Monte Carlo simulation to forecast 15 years cumulative returns
dist_plot = MC_15_returns.plot_distribution()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.018353,0.990991,0.993542,1.004285,1.010543,0.996096,1.004661,1.009838,1.000822,0.996604,...,0.989897,1.000037,1.012813,1.001722,1.000656,1.002117,1.003528,1.002606,0.991949,0.997364
2,1.014560,0.992820,0.996145,1.002978,1.005147,1.004685,1.001202,1.010342,0.993041,0.992330,...,0.988645,1.007607,1.020869,1.005857,1.006082,0.996915,1.004331,0.990710,0.987952,0.991272
3,1.019269,1.001492,1.009462,1.014306,1.001689,1.025238,0.996099,1.011401,0.994015,0.994844,...,0.986177,1.005196,1.031488,1.005400,1.002427,0.999050,1.002731,0.979590,0.999434,0.989640
4,1.014859,1.003336,1.004606,1.019190,1.003226,1.018442,0.993964,1.005165,1.001041,1.002201,...,0.992091,1.001366,1.029699,1.021144,0.997660,1.006650,0.990625,0.980227,1.004842,0.984748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7556,13.864071,6.077511,11.731330,31.243365,30.053930,14.757036,2.210814,15.528460,7.255163,16.637307,...,5.822107,2.259562,3.783502,5.828729,4.720930,35.072331,14.912683,5.653658,5.648629,6.315486
7557,13.746201,6.059207,11.585533,31.287242,30.582049,14.673008,2.186417,15.436076,7.339485,16.766272,...,5.823840,2.253127,3.775175,5.815855,4.766911,35.030715,15.116486,5.676312,5.688687,6.379419
7558,13.599568,6.013279,11.615333,31.745904,30.424040,14.651556,2.171581,15.329036,7.283202,16.707132,...,5.856829,2.229323,3.777023,5.802119,4.840632,35.167994,15.167563,5.681892,5.727080,6.358350
7559,13.777501,5.987253,11.684295,31.652897,30.570575,14.567353,2.180296,15.222299,7.293879,16.740933,...,5.890849,2.237878,3.812925,5.792500,4.856454,34.870746,15.323380,5.679676,5.710282,6.373808


### Fund Analysis

In [21]:
# Summary statistics from the Monte Carlo simulation results
tbl_15 = MC_15_returns.summarize_cumulative_return()

# Print summary statistics
print(tbl_15)

count           500.000000
mean              9.200762
std               6.659594
min               1.045214
25%               4.941116
50%               7.381182
75%              11.327780
max              50.993592
95% CI Lower      2.296975
95% CI Upper     26.810558
Name: 7560, dtype: float64


### Calculate the expected portfolio return at the 95% lower and upper confidence intervals based on a `$20,000` initial investment.

In [22]:
# Set initial investment
init_invst = 30000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_15 = round(tbl_15[-2] * init_invst,2)
ci_upper_15 = round(tbl_15[-1] * init_invst,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${init_invst} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower_15} and ${ci_upper_15}")

There is a 95% chance that an initial investment of $20000 in the portfolio over the next 30 years will end within in the range of $45939.5 and $536211.17
